In [1]:
from pykrx import stock

import pandas as pd
import numpy as np

from pytimekr import pytimekr
import datetime
from datetime import timedelta

import time
import pickle

In [2]:
stock_data = pd.read_csv('C:/Users/kim56/anaconda3/Find-A/quant/Data/mom_data.csv', index_col = 0, parse_dates=['날짜'], dtype = {'티커' : 'str'}).dropna(subset=['티커'])
stock_data


KeyboardInterrupt



In [3]:
grp = stock_data.groupby(by=['종목명', '티커']).count().reset_index()
for name in stock_data.종목명.unique() : 
    if len(grp[grp['종목명'] == name]) > 1 : 
        print(name)

원익피앤이


In [4]:
grp[grp.종목명 == '원익피앤이'] #같은 종목명, 다른 티커 => 제거해줌.

,종목명,티커,날짜,시가,고가,저가,종가,거래량,등락률,ABS_MOM
1587,원익피앤이,131390,2753,2753,2753,2753,2753,2753,2752,2752
1588,원익피앤이,217820,1738,1738,1738,1738,1738,1738,1737,1737


In [5]:
stock_data = stock_data.loc[stock_data.종목명 != '원익피앤이']
stock_data

,티커,종목명,날짜,시가,고가,저가,종가,거래량,등락률,ABS_MOM
0,060310,3S,2003-01-02,1037.0,1107.0,1007.0,1081.0,164270.0,7.135778,0.000000
1,060310,3S,2003-01-03,1095.0,1173.0,1095.0,1111.0,238965.0,2.775208,-2.180285
2,060310,3S,2003-01-06,1114.0,1132.0,1075.0,1119.0,144530.0,0.720072,-2.823614
3,060310,3S,2003-01-07,1138.0,1187.0,1095.0,1111.0,168800.0,-0.714924,-3.193958
4,060310,3S,2003-01-08,1101.0,1126.0,1074.0,1086.0,132570.0,-2.250225,-3.783407
...,...,...,...,...,...,...,...,...,...,...
4942,238490,힘스,2022-12-23,6050.0,6050.0,5950.0,6020.0,9421.0,-0.495868,-0.335925
4943,238490,힘스,2022-12-26,5940.0,6070.0,5860.0,5860.0,24368.0,-2.657807,-2.491430
4944,238490,힘스,2022-12-27,5940.0,5990.0,5860.0,5950.0,7276.0,1.535836,1.698196
4945,238490,힘스,2022-12-28,5880.0,5970.0,5870.0,5940.0,6909.0,-0.168067,0.008601


In [3]:
# 날짜 index 생성을 위한 함수 정의
def get_str_date_range(start, end, freq = 'BM'): # BM : 주말이 아닌 평일 중 각 달의 가장 마지막 날
    dr = list(pd.date_range(start, end, freq = freq))
    for i in range(len(dr)):
        date = dr[i].date()
        # 거래소가 개장하지 않는 경우에는 그 전날, 혹은 그 전전날의 주가 데이터를 가져옴.
        # ex. 공휴일, 주말, 경우에 따라 12월 30일이나 31일
        while (date in pytimekr.holidays(year=date.year)) or (date.weekday() >= 5) or (date.month == 12 and date.day >= 30) : 
            date = date - timedelta(days = 1)
        dr[i] = date.strftime("%Y%m%d")
    return dr

In [4]:
START = '20030101'
END = '20221231'

stock_info = pd.DataFrame(stock_data.티커.unique(), stock_data.종목명.unique())
stock_info

,0
AK홀딩스,006840
BYC,001460
BYC우,001465
CJ,001040
CJ대한통운,000120
...,...
휴스틸,005010
흥국화재,000540
흥국화재2우B,000547
흥국화재우,000545


In [10]:
date_range = get_str_date_range(START, END)
year_ends = [date for i, date in enumerate(date_range) if (i+1) % 12 == 0]

In [19]:
stock.get_market_cap(year_ends[0]).reset_index().iloc[:200][['티커', '시가총액']]

,티커,시가총액
0,005930,66375411960000
1,017670,16085097000500
2,060000,14632491546000
3,005490,13789754025000
4,015760,13762168834000
...,...,...
195,039870,163894510320
196,066270,163800000000
197,005880,162000000000
198,028050,160200000000


In [9]:
# kospi200 = stock.get_market_cap(year_ends[0]).reset_index().iloc[:200]
# kospi200['날짜'] = pd.to_datetime(year_ends[0])
# kospi200


def filter_kospi200(start, end, stock_data) : 
    date_range = get_str_date_range(start, end)
    year_ends = [date for i, date in enumerate(date_range) if (i+1) % 12 == 0]
    for i in range(len(year_ends)) : 
        date = year_ends[i]
        if date[:4] == '2006' or date[:4] == '2017' : 
            year_ends[i] = str(int(date) - 1)
    
    results = []
    
    for i in np.arange(len(year_ends)) : 
        
        if i == 0 : 
            date0 = start
        else : 
            date0 = year_ends[i-1]
        date1 = year_ends[i]
        cap = stock.get_market_cap(date).reset_index()
        ticker_index = cap.iloc[:200]['티커']
        df = stock_data.loc[(stock_data['날짜'] >= date0) & (stock_data['날짜'] <= date1) & (stock_data['티커'].isin(ticker_index)) ]
        results.append(df)
    return results

In [27]:
stock_data = pd.read_pickle('C:/Users/kim56/anaconda3/Find-A/quant/Data/fundamental_data.p')
stock_data

,티커,종목명,날짜,시가,고가,저가,종가,거래량,등락률,ABS_MOM,BPS,PER,PBR,시가총액
0,095570,AJ네트웍스,2015-08-21,6180.0,7300.0,5600.0,7100.0,1994022.0,NaN,NaN,NaN,NaN,NaN,NaN
1,095570,AJ네트웍스,2015-08-24,6700.0,6940.0,6240.0,6480.0,392831.0,-8.732394,0.000000,NaN,NaN,NaN,NaN
2,095570,AJ네트웍스,2015-08-25,6610.0,6730.0,6189.0,6220.0,242730.0,-4.012346,2.360024,NaN,NaN,NaN,NaN
3,095570,AJ네트웍스,2015-08-26,6260.0,7760.0,6260.0,7090.0,703721.0,13.987138,13.573006,NaN,NaN,NaN,NaN
4,095570,AJ네트웍스,2015-08-27,7239.0,8060.0,6960.0,7570.0,499913.0,6.770099,4.766974,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3785310,003280,흥아해운,2022-12-23,1505.0,1510.0,1450.0,1455.0,252745.0,-3.960396,-3.803694,428.0,13.77,3.80,3.906905e+11
3785311,003280,흥아해운,2022-12-26,1440.0,1445.0,1380.0,1410.0,226158.0,-3.092784,-2.906875,428.0,13.77,3.80,3.906905e+11
3785312,003280,흥아해운,2022-12-27,1410.0,1430.0,1375.0,1390.0,211506.0,-1.418440,-1.238906,428.0,13.77,3.80,3.906905e+11
3785313,003280,흥아해운,2022-12-28,1390.0,1410.0,1360.0,1385.0,146629.0,-0.359712,-0.171470,428.0,13.77,3.80,3.906905e+11


In [32]:
def filter_kospi200(start, end, stock_data) : 
    
    """
    이전 분기 시가총액 데이터를 활용하여 시가총액 상위 200위 이상의 종목으로 한정하여 스크리닝 진행."""
    
    date_range = get_str_date_range(start, end)
    quarter_ends = [date for i, date in enumerate(date_range) if (i+1) % 3 == 0]
    for i in range(len(quarter_ends)) : 
        date = quarter_ends[i]
        if date[:4] == '2006' or date[:4] == '2017' : 
            quarter_ends[i] = str(int(date) - 1)
    
    results = []
    
    for i in np.arange(len(quarter_ends)) : 
        if i == 0 : 
            continue
        else : 
            date0 = quarter_ends[i-1]
            date1 = quarter_ends[i]
            pool = stock_data.loc[stock_data.날짜 == date0].sort_values(by = '시가총액', ascending = False)
            ticker_index = pool.iloc[:200]['티커']
            df = stock_data.loc[(stock_data['날짜'] >= date0) & (stock_data['날짜'] <= date1) & (stock_data['티커'].isin(ticker_index)) ]
            results.append(df)
    return pd.concat(results)

In [31]:
results = filter_kospi200(START, END, stock_data)
results

,티커,종목명,날짜,시가,고가,저가,종가,거래량,등락률,ABS_MOM,BPS,PER,PBR,시가총액
1872,006840,AK홀딩스,2003-03-31,6980.0,7011.0,6729.0,6731.0,2960.0,-3.581149,-3.080381,34409.0,7.11,0.31,6.450000e+10
1873,006840,AK홀딩스,2003-04-01,6824.0,6824.0,6635.0,6668.0,2400.0,-0.935968,-0.428292,34409.0,7.11,0.31,6.450000e+10
1874,006840,AK홀딩스,2003-04-02,6636.0,6855.0,6636.0,6856.0,5670.0,2.819436,3.275126,34409.0,7.11,0.31,6.450000e+10
1875,006840,AK홀딩스,2003-04-03,7074.0,7106.0,6980.0,7106.0,9820.0,3.646441,4.039021,34409.0,7.11,0.31,6.450000e+10
1876,006840,AK홀딩스,2003-04-04,7106.0,7324.0,6887.0,7169.0,6240.0,0.886575,1.259774,34409.0,7.11,0.31,6.450000e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3752955,081660,휠라홀딩스,2022-12-23,34100.0,35000.0,33900.0,34450.0,360537.0,1.472754,1.464576,26806.0,8.44,1.23,2.007861e+12
3752956,081660,휠라홀딩스,2022-12-26,34450.0,34600.0,34000.0,34500.0,138323.0,0.145138,0.132558,26806.0,8.44,1.23,2.007861e+12
3752957,081660,휠라홀딩스,2022-12-27,34500.0,35000.0,34300.0,34950.0,316271.0,1.304348,1.273898,26806.0,8.44,1.23,2.007861e+12
3752958,081660,휠라홀딩스,2022-12-28,34050.0,34250.0,33500.0,33550.0,180426.0,-4.005722,-4.021987,26806.0,8.44,1.23,2.007861e+12


In [34]:
results.to_pickle('C:/Users/kim56/anaconda3/Find-A/quant/Data/kospi200_data.p')